In [7]:
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import random

In [8]:
def rand_samples(m,b,num,rand_param):
    x_axis,y_axis,labels=np.array([]), np.array([]), np.array([])
    if(m>=0):
        c=1
    else:
        c=-1
    pos_num = int(num / 2)
    neg_num = num-pos_num
    
    for state,num in[['pos', pos_num], ['neg', neg_num]]:
        x = np.random.randint(0, rand_param,num)
        r = np.random.randint(1, rand_param,num)
        y_line=m*x+b
        if state == 'pos':
            y=m*x+b-(r*c)
            for i in range(len(y)):
                for j in range(len(y_line)):
                    if(y[i]==y_line[j]):
                        y-=1
            labels = np.append(labels, np.ones(num, dtype=int))
        else:
            y=m*x+b+(r*c)
            for i in range(len(y)):
                for j in range(len(y_line)):
                    if(y[i]==y_line[j]):
                        y+=1
            labels = np.append(labels,-1*np.ones(num, dtype=int))
        x_axis=np.append(x_axis,x)
        y_axis=np.append(y_axis,y)
    for z in range(len(labels)):
            if(z%20==0):
                labels[z]*=-1              
        
    return x_axis,y_axis,labels
            

In [9]:
def Display(dataset,w0,w1,c,rand_param,pos_num): 
    plt.scatter(dataset[:pos_num,0], dataset[:pos_num,1], color='blue', marker='o', label='Positive') 
    plt.scatter(dataset[pos_num:,0], dataset[pos_num:,1], color='red', marker='o', label='Negative')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend(loc='upper left')
    x=np.arange(rand_param+1)
    a=-w0/w1
    b1=-c/w1
    y=a*x+b1
    plt.plot(x,y,'y')
    plt.show()
def PLA(dataset,rand_param,pos_num):
    starttime = time.time() 
    W = np.zeros(2)
    c = 0           
    count = 0
    while True:
        count += 1
        cease = True
        for i in range(0,len(dataset)):
            xn = dataset[i][:2]    
            Xn = np.array(xn)
            Y = np.dot(W,Xn) + c
            if np.sign(Y) == np.sign(dataset[i][2]):  
                continue
            else:
                cease = False
                W = W + (dataset[i][2]) * Xn
                c = c + dataset[i][2]
        if cease:
            break
    endtime = time.time()
    dtime = endtime - starttime
    print("W:",W)
    print("time: %.8s s" % dtime)
    print("round:",count)
    Display(dataset,W[0],W[1],c,rand_param,pos_num)
    return W

In [10]:
def num_fault(W,b,dataset): 
    count = 0
    for i in range(0,len(dataset)):
        X = np.array(dataset[i][:-1])    
        Y = np.dot(W,X) + b
        if np.sign(Y) == np.sign(dataset[i][-1]):
            continue
        else:
            count += 1
    return count

def POCKET_PLA(dataset,num,rand_param,pos_num):
    starttime = time.time()
    count = 0       
    W = np.ones(2)
    b = 0
    best_W = W      
    best_b = b      
    min_num = num   
    while True:
        count += 1
        cease = True
        faultset = [] 
        for i in range(0,len(dataset)):
            x = dataset[i][:-1]    
            X = np.array(x)
            Y = np.dot(W,X) + b
            if np.sign(Y) == np.sign(dataset[i][-1]):
                continue
            else:
                cease = False
                faultset.append(dataset[i])
        if cease == False:
            j = random.randint(0,len(faultset)-1) 
            W = W + (faultset[j][-1]) * faultset[j][:-1]
            b = b + faultset[j][-1]
            num = num_fault(W,b,dataset)
            if num < min_num: 
                count = 0    
                min_num = num
                best_W = W
                best_b = b
        if (cease or count==100):  
            break
    endtime = time.time()
    dtime = endtime - starttime
    print("time: %.8s s" % dtime)
    print("best_W:",best_W)
    print("best_b:",best_b)
    print("count:",count)
    print("min_fault_point:",min_num)
    Display(dataset,best_W[0],best_W[1],best_b,rand_param,pos_num)
    return best_W


In [ ]:
def main():
    m,b=2,1
    num=2000
    rand_param=2000
    pos_num = int(num / 2)
    x_axis,y_axis,labels=rand_samples(m,b,num,rand_param)
    df=pd.DataFrame({'x':x_axis,'y':y_axis,'labels':labels},dtype="float64")
    dataset=df.values
    W = PLA(dataset,rand_param,pos_num)
    B_W = POCKET_PLA(dataset,num,rand_param,pos_num)
    
if __name__ == '__main__':
    main()
